In [1]:
import pandas as pd
# Path to the CSV file
file_path = 'dataset_ruangan.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(file_path)

df.head()

,ID,Kode,Nama,Lantai,Fakultas,Status,Kapasitas,Luas,Rating,Tujuan
0,739,NaN,Ruang Scienfic Lecture Hall,1,TPB,Tidak Aktif,500,750.00,4.2,Sidang
1,736,103,NaN,1,TPB,Tidak Aktif,210,0.00,4.3,Kelas
2,227,303,AULA,3,Fakultas Keguruan dan Ilmu Pendidikan,Aktif,200,218.90,3.9,Seminar
3,83,102,NaN,1,Fakultas Teknik,Tidak Aktif,200,1108.42,3.5,Sidang
4,193,NaN,Ruang Sidang Senat,3,Fakultas Keguruan dan Ilmu Pendidikan,Aktif,200,296.00,3.9,Seminar


import pandas as pd
# Path to the CSV file
file_path = r'C:\Users\user\Documents\ZENROOM\dataset_ruangan.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(file_path)

df.head()

In [2]:
# Hapus ruangan dengan status 'Tidak Aktif'
cleaned_data = df[df['Status'] == 'Aktif']

# Preview cleaned data
print(cleaned_data)

      ID        Kode                    Nama  Lantai  \
2    227         303                    AULA       3   
4    193         NaN      Ruang Sidang Senat       3   
7    507        B3-E                 Kelas E       3   
8    510        B2-D                 Kelas D       2   
9    577     ATB.3.1                 ATB.3.1       3   
..   ...         ...                     ...     ...   
875  374       B.1.6      R. WAKIL DEKAN III       1   
876  375       B.1.4                  R. KTU       1   
878  380      B.1.11         R. PERLENGKAPAN       1   
879  384     B.2.2.1  R. KASUB KEMAHASISWAAN       2   
894  347  A.D.1.Lab1            Lab Komputer       1   

                                          Fakultas Status  Kapasitas   Luas  \
2            Fakultas Keguruan dan Ilmu Pendidikan  Aktif        200  218.9   
4            Fakultas Keguruan dan Ilmu Pendidikan  Aktif        200  296.0   
7                              Fakultas Kedokteran  Aktif        160  220.0   
8          

In [3]:
# Urutkan berdasarkan fakultas
df_sorted = cleaned_data.sort_values(by='Fakultas')

print(df_sorted)

      ID          Kode          Nama  Lantai          Fakultas Status  \
369  311       A.C.2.2  Ruang Kuliah       2  Fakultas Ekonomi  Aktif   
260  336  A.D.1.4 (D3)  Ruang Kuliah       1  Fakultas Ekonomi  Aktif   
265  328  A.D.3.2 (D3)  Ruang Kuliah       3  Fakultas Ekonomi  Aktif   
267  306       A.B.3.1  Ruang Kuliah       3  Fakultas Ekonomi  Aktif   
273  878      A.B.2.2B  Ruang Kuliah       2  Fakultas Ekonomi  Aktif   
..   ...           ...           ...     ...               ...    ...   
174  721           307       GKB-307       3               TPB  Aktif   
173  217           202       GKB-202       2               TPB  Aktif   
171  720           306       GKB-306       3               TPB  Aktif   
190  712           205       GKB-205       2               TPB  Aktif   
199  729           501       GKB-501       5               TPB  Aktif   

     Kapasitas   Luas  Rating   Tujuan  
369         40    NaN     4.1    Rapat  
260         50    NaN     3.6   Sidang  


In [4]:
df_sorted.replace('NA', pd.NA, inplace=True)

# Menghapus baris dengan nilai NaN
df_cleaned = df_sorted.dropna()

# Menampilkan DataFrame yang sudah dibersihkan
print(df_cleaned)

      ID          Kode                             Nama  Lantai  \
570  275  A.A.3.Lab. 2                    Lab. Komputer       3   
571  292       A.C.3.1                     Ruang Kuliah       3   
667  893       A.E.1.2  Ruang Kuliah, seminar dan Ujian       1   
894  347    A.D.1.Lab1                     Lab Komputer       1   
380   43    C1_R.MIH A                Ruang Kelas MIH A       1   
..   ...           ...                              ...     ...   
176  717           303                          GKB-303       3   
174  721           307                          GKB-307       3   
171  720           306                          GKB-306       3   
190  712           205                          GKB-205       2   
199  729           501                          GKB-501       5   

             Fakultas Status  Kapasitas    Luas  Rating   Tujuan  
570  Fakultas Ekonomi  Aktif         20    0.00     4.1    Rapat  
571  Fakultas Ekonomi  Aktif         20   66.75     4.4    Ra

In [5]:
# Opsional: Menyimpan DataFrame yang sudah dibersihkan ke file CSV
df_cleaned.to_csv('nama_file_bersih.csv', index=False)

# CONTENT BASED FILTERING

In [6]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

In [11]:
df_cleaned = df.copy()
df_cleaned.loc[:, 'combined_features'] = df_cleaned.apply(lambda row: f"{row['Lantai']} {row['Fakultas']} {row['Kapasitas']} {row['Rating']} {row['Tujuan']}", axis=1)

In [12]:
# Inisialisasi CountVectorizer untuk mengubah teks menjadi vektor fitur
count = CountVectorizer()
count_matrix = count.fit_transform(df_cleaned['combined_features'])

# Menghitung matriks kemiripan kosinus berdasarkan vektor fitur
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [13]:
# Fungsi untuk mendapatkan rekomendasi berdasarkan kesamaan kosinus
def get_content_based_recommendations(room_id, df_cleaned, cosine_sim, top_n=5):
    idx = df_cleaned.index[df_cleaned['ID'] == room_id].tolist()[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]  
    room_indices = [i[0] for i in sim_scores]
    return df_cleaned.iloc[room_indices]

In [14]:
# Mendapatkan rekomendasi untuk ruangan dengan ID 227
room_id = 227
recommendations = get_content_based_recommendations(room_id, df_cleaned, cosine_sim)
print("Content-Based Recommendations:")
print(recommendations[['ID', 'Nama', 'Lantai', 'Fakultas', 'Kapasitas', 'Luas', 'Rating', 'Tujuan']])

Content-Based Recommendations:
     ID                     Nama  Lantai  \
4   193       Ruang Sidang Senat       3   
41  818         PENDIDIKAN KIMIA       1   
75  809               S1 PG-PAUD       1   
85  264  S1 PENDIDIKAN SOSIOLOGI       1   
91  231             Ruang Kuliah       3   

                                 Fakultas  Kapasitas    Luas  Rating   Tujuan  
4   Fakultas Keguruan dan Ilmu Pendidikan        200  296.00     3.9  Seminar  
41  Fakultas Keguruan dan Ilmu Pendidikan         80   92.16     3.7  Seminar  
75  Fakultas Keguruan dan Ilmu Pendidikan         70  117.39     4.1  Seminar  
85  Fakultas Keguruan dan Ilmu Pendidikan         60  100.78     3.5  Seminar  
91  Fakultas Keguruan dan Ilmu Pendidikan         60   94.71     3.7  Seminar  
